In [1]:
import JSON
using Knet
import .Iterators: cycle, Cycle, take, repeat

In [2]:
trn = open("snli_1.0_train.jsonl", "r")
tst = open("snli_1.0_test.jsonl", "r")
val = open("snli_1.0_dev.jsonl", "r")

IOStream(<file snli_1.0_dev.jsonl>)

In [3]:
tstx1 = []
tstx2 = []
tsty = []

trnx1 = []
trnx2 = []
trny = []

valx1 = []
valx2 = []
valy = []

0-element Array{Any,1}

In [4]:
function take_best_randomly(x)
    n = 0
    e = 0
    c = 0
    for i in x
        if i == "neutral"; n+=1; end
        if i == "entailment"; e+=1; end
        if i == "contradiction"; c+=1; end
    end
    if n>e; return rand(["neutral", "contradiction"]); end
    if e>c; return rand(["entailment", "neutral"])
    else 
        return rand(["entailment", "contradiction"])
    end
end

take_best_randomly (generic function with 1 method)

In [5]:
for i in 1:10000
    d1 = JSON.parse(tst)
    if d1["gold_label"] == "-"; continue; end;
    push!(tstx1, d1["sentence1"])
    push!(tstx2, d1["sentence2"])
    push!(tsty, d1["gold_label"])
end

for i in 1:10000
    d1 = JSON.parse(val)
    gold = d1["gold_label"]
    if gold == "-"; gold = take_best_randomly(d1["annotator_labels"]); end;
    push!(valx1, d1["sentence1"])
    push!(valx2, d1["sentence2"])
    push!(valy, gold)
    end


for i in 1:550152
    d1 = JSON.parse(trn)
    gold = d1["gold_label"]
    if gold == "-"; gold = take_best_randomly(d1["annotator_labels"]); end;
    push!(trnx1, d1["sentence1"])
    push!(trnx2, d1["sentence2"])
    push!(trny, gold)
end

In [6]:
size(trnx1) == size(trnx2) == size(trny), size(tstx1) == size(tstx2) == size(tsty), size(valx1) == size(valx2) == size(valy)

(true, true, true)

In [7]:
f = open("glove.42B.300d.txt")
lines = readlines(f);

In [8]:
embeddingdict = Dict()
embeddingindex = 1
for line in lines
    strword = split(line)
    strname = strword[1]
    embeddingdict[strname] = embeddingindex
    embeddingindex+=1
end

In [9]:
#split strings into lists of words
tstx1 = (x -> split(x)).(tstx1)
tstx2 = (x -> split(x)).(tstx2)
trnx1 = (x -> split(x)).(trnx1)
trnx2 = (x -> split(x)).(trnx2)
valx1 = (x -> split(x)).(valx1)
valx2 = (x -> split(x)).(valx2)
#make everything lowercase
tstx1 = [(x -> lowercase(x)).(s) for s in tstx1]
tstx2 = [(x -> lowercase(x)).(s) for s in tstx2]
trnx1 = [(x -> lowercase(x)).(s) for s in trnx1]
trnx2 = [(x -> lowercase(x)).(s) for s in trnx2]
valx1 = [(x -> lowercase(x)).(s) for s in valx1]
valx2 = [(x -> lowercase(x)).(s) for s in valx2];

In [10]:
function map_labels(x)
    if x == "entailment"; return 1; end
    if x == "neutral"; return 2; end
    if x == "contradiction"; return 3; end
    x
end

tsty = map_labels.(tsty)
trny = map_labels.(trny)
valy = map_labels.(valy);

In [11]:
#create vocab
vocab = Dict()

for sentence in vcat(tstx1,tstx2,trnx1,trnx2,valx1,valx2)
    for word in sentence
        vocab[word] = 1
    end
end
vocab["UNK"] = 1
length(vocab)

57364

In [12]:
#create dictionaries 
w2i = Dict()
i2w = Dict()
dictindex = 1

for word in vocab
    w2i[word[1]] = dictindex
    i2w[dictindex] = word[1]
    dictindex+=1
end

In [13]:
#Find maxlength for padding
maxlength = 0
for i in vcat(tstx1,tstx2,trnx1,trnx2,valx1,valx2)
    if length(i) > maxlength; maxlength = length(i); end;
end
maxlength

78

In [14]:
#map words to integers
tstx1 = [(x -> w2i[x]).(s) for s in tstx1]
tstx2 = [(x -> w2i[x]).(s) for s in tstx2]
trnx1 = [(x -> w2i[x]).(s) for s in trnx1]
trnx2 = [(x -> w2i[x]).(s) for s in trnx2]
valx1 = [(x -> w2i[x]).(s) for s in valx1]
valx2 = [(x -> w2i[x]).(s) for s in valx2];

In [15]:
#PADDING
for sentence in vcat(tstx1,tstx2,trnx1,trnx2,valx1,valx2)
    while length(sentence) != maxlength
        pushfirst!(sentence,w2i["UNK"])
    end
end

In [16]:
#note to self: word is (word, 1) so take [1]
embedmatrix = []
no_embeddings = []
count = 0
for word in vocab
    if word[1] in keys(embeddingdict)
        wordvector = (x-> parse(Float32, x)).(split(lines[embeddingdict[word[1]]])[2:301])
        count += 1
    else
        wordvector = xavier(Float32, 300)
        push!(no_embeddings, (word[1], wordvector))
    end
    push!(embedmatrix, hcat(wordvector))
end

In [17]:
embedmatrix = hcat(embedmatrix...);

In [18]:
size(embedmatrix)

(300, 57364)

In [19]:
println(count, " out of ", length(vocab), " words are in Stanford Glove Embeddings. The rest is initialized randomly with xavier.")

33319 out of 57364 words are in Stanford Glove Embeddings. The rest is initialized randomly with xavier.


MODEL

In [20]:
# Set constants for the model and training
EPOCHS=3                  # Number of training epochs
BATCHSIZE=16               # Number of instances in a minibatch
EMBEDSIZE=300             # Word embedding size
NUMHIDDEN=100             # Hidden layer size
MAXLEN=150                # maximum size of the word sequence, pad shorter sequences, truncate longer ones
VOCABSIZE=length(vocab)   # maximum vocabulary size, keep the most frequent 30K, map the rest to UNK token
NUMCLASS=3                # number of output classes
DROPOUT=0.5               # Dropout rate
LR=0.002                  # Learning rate
BETA_1=0.9                # Adam optimization parameter
BETA_2=0.999              # Adam optimization parameter
EPS=1e-08                 # Adam optimization parameter

1.0e-8

In [21]:
tstx = []
trnx = []
valx = []
for i in 1:length(tstx1)
    push!(tstx, (tstx1[i], tstx2[i]))
end
for i in 1:length(trnx1)
    push!(trnx, (trnx1[i], trnx2[i]))
end
for i in 1:length(valx1)
    push!(valx, (valx1[i], valx2[i]))
end

In [22]:
dtrn = minibatch(trnx,trny,BATCHSIZE;shuffle=true)
dtst = minibatch(tstx,tsty,BATCHSIZE)
dval = minibatch(valx,valy,BATCHSIZE);

In [23]:
#model struct
struct LSTMN
    embeds
    lstm1
    lstm2
    Wy
    Wh
    Wp
    Wx
    W1
    output
end

In [24]:
#model constructor
function LSTMN(input::Int, embed::Int, hidden::Int, output::Int; pdrop=0)
    embeds = param(KnetArray(embedmatrix))
    lstm1 = RNN(embed,hidden)
    lstm2 = RNN(embed,hidden)
    Wy = param(hidden,hidden)
    Wh = param(hidden,hidden)
    Wp = param(hidden,hidden)
    Wx = param(hidden,hidden)
    W1 = param(1,hidden)
    output = param(output,2*hidden)
    LSTMN(embeds, lstm1, lstm2, Wy, Wh, Wp, Wx, W1, output)
end

LSTMN

In [40]:
function(lstmn::LSTMN)(input)
    input1 = (x->x[1]).(input)
    input2 = (x->x[2]).(input)
    
    lstmn.lstm1.h = 0
    lstmn.lstm1.c = 0
    lstmn.lstm2.h = 0
    lstmn.lstm2.c = 0
    
    embed1 = lstmn.embeds[:, permutedims(hcat(input1...))]
    embed1 = dropout(embed1, rand([0.1,0.2,0.3,0.4]))
    hidden1 = lstmn.lstm1(embed1)
    hidden1 = dropout(hidden1, rand([0.1,0.2,0.3,0.4]))
        
    lstmn.lstm2.h = lstmn.lstm1.h
    lstmn.lstm2.c = lstmn.lstm1.c

    embed2 = lstmn.embeds[:, permutedims(hcat(input2...))]
    embed2 = dropout(embed2, rand([0.1,0.2,0.3,0.4]))
    hidden2 = lstmn.lstm2(embed2)
    hidden2 = dropout(hidden2, rand([0.1,0.2,0.3,0.4]))
     
    println("hidden1: ", size(hidden1))
    
    #(K,K) * (K,B,L) => (K,K) * (K,B*L) = (K,B*L) => reshape => (K,B,L)
    Y = reshape(hidden1, NUMHIDDEN, BATCHSIZE*78)
    println("Y: ",size(Y))
    M1 = lstmn.Wy * Y
    M1 = reshape(M1, NUMHIDDEN, BATCHSIZE, 78)
    println("M1: ", size(M1))
    M2 = lstmn.Wh * hidden2[:,:,end]
    println("M2: ", size(M2))
    M = M1 .+ M2
    println("M: ", size(M))
    
    #calculate alfa
    # reshape M (100,16,78) -> (100, 16*78)
    M = reshape(M, NUMHIDDEN, BATCHSIZE*78)
    alfa = lstmn.W1 * M
    alfa = reshape(alfa, BATCHSIZE, 78)
    alfa = softmax(alfa; dims=1)
    println("alfa: ", size(alfa))
   
    #calculate r
    Y = reshape(hidden1, NUMHIDDEN*BATCHSIZE, 78)
    println("Y: ",size(Y))
    r = Y * alfa'
    println("r: ", size(r))
    
    r = reshape(r, NUMHIDDEN, BATCHSIZE)
    #r = 
    
    
    return lstmn.output * hidden2
end

(l::LSTMN)(input,output) = nll(l(input), vcat(output...))
(l::LSTMN)(d::Knet.Data) = Knet.mean(l(x,y) for (x,y) in d)

In [41]:
x1, y1 = first(dtrn)
model(x1,y1)

hidden1: (100, 16, 78)
Y: (100, 1248)
M1: (100, 16, 78)
M2: (100, 16)
M: (100, 16, 78)
alfa: (16, 78)
Y: (1600, 78)
r: (1600, 16)


DimensionMismatch: DimensionMismatch("")

In [31]:
model = LSTMN(VOCABSIZE,EMBEDSIZE,NUMHIDDEN,NUMCLASS,pdrop=DROPOUT)

LSTMN(P(KnetArray{Float32,2}(300,57364)), LSTM(input=300,hidden=100), LSTM(input=300,hidden=100), P(KnetArray{Float32,2}(100,100)), P(KnetArray{Float32,2}(100,100)), P(KnetArray{Float32,2}(100,100)), P(KnetArray{Float32,2}(100,100)), P(KnetArray{Float32,2}(1,100)), P(KnetArray{Float32,2}(3,200)))

In [28]:
x1, y1 = first(dtrn)
model(x1,y1)

hidden1: (100, 16, 78)
Y: (100, 1248)
M1: (100, 16, 78)
M2: (100, 16)
M: (100, 16, 78)
alfa: (16, 78)
Y: (1600, 78)
r: (1600, 16)
r: (1600, 16)


MethodError: MethodError: no method matching *(::KnetArray{Float32,2}, ::KnetArray{Float32,3})
Closest candidates are:
  *(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:502
  *(!Matched::Knet.Bcasted{Float32}, ::KnetArray{Float32,N} where N) at /home/ec2-user/.julia/packages/Knet/05UDD/src/binary.jl:127
  *(!Matched::AutoGrad.Value{##395<:Number}, ::##396) where {##395<:Number, ##396} at none:0
  ...

In [29]:
function fasttrain!(lstmn::LSTMN, dtrn, dtst, max_iters=500)
    a = adam(lstmn, take(cycle(dtrn), max_iters+1);lr=LR,beta1=BETA_1,beta2=BETA_2,eps=EPS)
    progress!(a)
end

fasttrain! (generic function with 2 methods)

In [30]:
accuracy(model, dtst)

hidden1: (100, 16, 78)
Y: (100, 1248)
M1: (100, 16, 78)
M2: (100, 16)
M: (100, 16, 78)
alfa: (16, 78)
Y: (1600, 78)
r: (1600, 16)
r: (1600, 16)


MethodError: MethodError: no method matching *(::KnetArray{Float32,2}, ::KnetArray{Float32,3})
Closest candidates are:
  *(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:502
  *(!Matched::Knet.Bcasted{Float32}, ::KnetArray{Float32,N} where N) at /home/ec2-user/.julia/packages/Knet/05UDD/src/binary.jl:127
  *(!Matched::AutoGrad.Value{##395<:Number}, ::##396) where {##395<:Number, ##396} at none:0
  ...